<a href="https://colab.research.google.com/github/gabrielyuto/Exercises-object-oriented-programming/blob/master/Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise da base de dados da empresa Olist

O conjunto Olist é um conjunto de dados públicos de comércio eletrônico brasileiro de pedidos feitos na loja Olist. O conjunto de dados contém informações de cerca de 100 mil pedidos feitos entre 2016 a 2018 em vários marketplaces no Brasil. Suas características permitem visualizar um pedido a partir de múltiplas dimensões: desde o status do pedido, preço, desempenho de pagamento e frete até a localização do cliente, atributos do produto e, finalmente, avaliações escritas pelos clientes. Também lançamos um conjunto de dados de geolocalização que relaciona os códigos postais brasileiros às coordenadas de latitude/longitude.

Estes são dados comerciais reais, que foram anonimizados e as referências às empresas e parceiros no texto da avaliação foram substituídas pelos nomes das grandes casas de Game of Thrones.



## Descompactando os arquivos

O arquivo compactado `archive.zip` contém os dados divididos em tabelas. Coloque-o na mesma pasta do notebook, ou se estiver usando o google colab, faça o upload do arquivo. A célua a seguir faz a descompatação dos dados

In [ ]:
#!unzip archive.zip

## Bibliotecas

A biblioteca `pandas` é bastante útil para carregar e manipular os dados.
A biblioteca `geopy` nos permite calcular distâncias a partir das coordenadas cartesianas (latitude e longitude)

In [ ]:
#!pip install geopy

In [ ]:
import pandas as pd
import geopy.distance as gp

## Tabela de Pedidos

Vamos começar analisando a tabela de pedidos. Ela se chama `olist_orders_dataset.csv`.

In [ ]:
df_order = pd.read_csv("olist_orders_dataset.csv")

Vamos observar as primeiras linhas da base de dados

In [ ]:
df_order.head()

Vamos olhar a frequência status dos pedidos:

In [ ]:
df_order['order_status'].value_counts()

E graficamente:

In [ ]:
df_order['order_status'].value_counts().plot(kind='bar')

Como esperado, o número de pedidos já entregues é muito maior que os demais. Vamos usar a eslaca logarítmica para visualizar

In [ ]:
df_order['order_status'].value_counts().plot(kind='bar',logy=True)

## Tempo para entrega

Podemos calcular o tempo de entrega fazendo a diferença entre a data da entrega e a data da compra. O `pandas` consegue fazer operações que envolvam datas e horas com facilidade. Entretanto, o `pandas` pode ter carregado as colunas em formato texto (`string`). Podemos converter para o formato de data e hora fazendo:

In [ ]:
df_order['order_purchase_timestamp'] = pd.to_datetime(df_order['order_purchase_timestamp'])

A coluna acima a conversão de uma coluna de cada vez. Como temos mais que uma podemos fazer mais de uma conversão ao mesmo tempo selecionando quais são as colunas que envolvam data e hora:

In [ ]:
df_order = df_order.assign(**df_order[['order_purchase_timestamp',
                           'order_approved_at',
                           'order_delivered_carrier_date',
                           'order_delivered_customer_date',
                           'order_estimated_delivery_date'
                          ]]
                        .apply(pd.to_datetime))

Agora podemos calcular a difereça entre a entrega e a compra:

In [ ]:
df_order['deliver_time'] = df_order['order_delivered_customer_date'] - df_order['order_purchase_timestamp']

Vamos calcular as estatísticas descritivas para ter uma ideia do tempo de entrega

In [ ]:
df_order['deliver_time'].describe()

O pandas não consegue criar alguns gráficos com diferença de tempo (`timedelta`). Uma alternativa é, por exemplo, conveter para dias. Na célula a seguir, fazemos um boxplot do número de dias:

In [ ]:
df_order['deliver_time'].astype('timedelta64[D]').plot(kind='box')

E na célula seguinte, um gráfico da densidade usando o histograma normalizado e uma estimativa não paramétrica (`kde`)

In [ ]:
df_order['deliver_time'].astype('timedelta64[D]').plot(kind='hist', density=True, logx=True,bins=50)
df_order['deliver_time'].astype('timedelta64[D]').plot(kind='kde', logx=True,xlim=[1,210])

Será que o tempo de entrega está relacionado do status do pedido? No gráfico a seguir, criamos um boxplot para cada tipo de status:

In [ ]:
(df_order
     .assign(**df_order[['deliver_time']].astype('timedelta64[D]'))
     .plot(kind='box',by='order_status',rot=90)
)

Vamos agora olhar o tempo estimado de entrega. Primeiramente, vamo verificar se o tempo estimado e o tempo real da entrega estão correlacionados. Para isso, vamos calcular o coeficiente de correlação e o criar o gráfico de dispersão (`scatter`). Para calcular o coeficiente de correlação, precisamos converter as datas para um valor numérico, o método que calcula a correlação não é diretamente aplicável a datas e horas:

In [ ]:
df_order.assign(**df_order[['order_estimated_delivery_date',
                            'order_delivered_customer_date']]
                .apply(pd.to_numeric)).corr()

In [ ]:
g = df_order.plot(x='order_estimated_delivery_date',y='order_delivered_customer_date',kind='scatter')
g.set_aspect(1)


Observando o gráfico e o coeficiente, vemos uma discrepância grande entre eles. O gráfico indica uma forte correlação, mas o coeficiente é proximo de zero. O que aconteceu?

Quando convertemos para numérico, o pandas converte para timestamp (o número de segundos desde 1o. de janeiro de 1970). Acontece que esses números são bastante grandes, e gera um problema numérico no cálculo da correlação. Por exemplo, veja as estatisticas descritivas:

In [ ]:
df_order.assign(**df_order[['order_estimated_delivery_date',
                            'order_delivered_customer_date']]
                .apply(pd.to_numeric))[['order_estimated_delivery_date',
                            'order_delivered_customer_date']].describe()

Vamos calcular a correlação de spearman (que converte os valores para o seu posto):

In [ ]:
df_order.assign(**df_order[['order_estimated_delivery_date',
                            'order_delivered_customer_date']]
                .apply(pd.to_numeric)).corr(method='spearman')

Como pode ser observado, o valor da correlação de Spearman é bem mais condizente com o gráfico

Vamos agora calcular o tempo estimado de entrega a partir do pedido:

In [ ]:
df_order['deliver_estimate'] = df_order['order_estimated_delivery_date'] - df_order['order_purchase_timestamp']

Como está distribuído o tempo estimado de entrega?

In [ ]:
df_order['deliver_estimate'].describe()

In [ ]:
df_order['deliver_estimate'].astype('timedelta64[D]').plot(kind='box')

O tempo estimado e o tempo de entrega estão correlacionados?

In [ ]:
df_order.assign(**df_order[['deliver_time',
                            'deliver_estimate']]
                .apply(pd.to_numeric)).corr(method='spearman')

In [ ]:
(df_order
     .assign(**df_order[['deliver_time','deliver_estimate']].astype('timedelta64[D]'))
     .plot(kind='scatter',x='deliver_time',y='deliver_estimate')
)

Será que a aprovação da compra influencia o tempo de entrega? Para isso, vamos calcular o prazao de entrega após a aprovação.

In [ ]:
df_order['deliver_after_approval_time'] = df_order['order_delivered_customer_date'] - df_order['order_approved_at']

E vamos comparar o prazo de entrega desde a compra com o prazo da entrega após a aprovação:

In [ ]:
df_order \
     .assign(**df_order[['deliver_time','deliver_after_approval_time']].astype('timedelta64[D]')) \
     .plot(kind='scatter',x='deliver_time',y='deliver_after_approval_time')


Quanto tempo demora para a aprovação de uma compra? Vamos calcular a diferença entre a aprovação e a compra

In [ ]:
df_order['approval_time'] = df_order['order_approved_at'] - df_order['order_purchase_timestamp']

e visualizar a distribuição:

In [ ]:
df_order['approval_time'].astype('timedelta64[D]').plot(kind='box')

e vamos reanalisar o tempo de aprovação com o tempo de entrega:

In [ ]:
df_order \
     .assign(**df_order[['deliver_time','approval_time']].astype('timedelta64[D]')) \
     .plot(kind='scatter',x='deliver_time',y='approval_time')

## Itens dos pedidos

Vamos agora olhar os itens que aparecem em cada pedido. Esses itens estão armazenados no arquivo `olist_order_items_dataset.csv`:

In [ ]:
df_orderm_item = pd.read_csv("olist_order_items_dataset.csv")

In [ ]:
df_orderm_item.head()

Os itens tem associado a data máxima para o envio. Vamos converter para o formato `datetime`

In [ ]:
df_orderm_item['shipping_limit_date'] = pd.to_datetime(df_orderm_item['shipping_limit_date'])

Quantos itens temos em cada compra? Como os itens do pedido são numerados, podemos pegar o `order_item_id` máximo de cada pedido como sendo o número de itens

In [ ]:
df_orderm_item.groupby('order_id')['order_item_id'].max().value_counts()

In [ ]:
df_orderm_item.groupby('order_id')['order_item_id'].max().value_counts().plot(kind='bar',logy=True)

Quais os items mais comprados? Como a base está anonimizada, não sabemos qual é, mas podemos ver a frequência de compras desses itens:

In [ ]:
df_orderm_item['product_id'].value_counts().head(30).plot(kind='bar')

Quais vendedores vendem mais?

In [ ]:
df_orderm_item['seller_id'].value_counts().head(30).plot(kind='bar')

Como variam os preços dos itens?

In [ ]:
df_orderm_item['price'].describe()

In [ ]:
df_orderm_item['price'].plot(kind='hist',logx=True,density=True,bins=300)
df_orderm_item['price'].plot(kind='kde', logx=True,xlim=[1,10000])

E do frete?

In [ ]:
df_orderm_item['freight_value'].plot(kind='hist',logx=True,density=True,bins=100)
df_orderm_item['freight_value'].plot(kind='kde', logx=True,xlim=[1,10000])

Agora vamos combinar dos dois conjuntos de dados. Podemos usar o número da ordem como o critério de junção dos dois conjuntos de dados.

In [ ]:
df = pd.merge(df_orderm_item, df_order, on = 'order_id')

Uma vez que combinados as duas tabelas, podemos calcular o tempo calcular o tempo máximo de envio a partir da data da compra:

In [ ]:
df['max_shipping_time'] = df['shipping_limit_date'] - df['order_purchase_timestamp']

E observar a sua distribuição:

In [ ]:
df['max_shipping_time'].astype('timedelta64[D]').plot(kind='box',logy=True)

## Informação de produtos

Vamos carregar a base de produtos para agregar mais informaçãoes às compras

In [ ]:
df_products = pd.read_csv("olist_products_dataset.csv")

In [ ]:
df_products.head()

Quais as categorias mais comuns?

In [ ]:
df_products['product_category_name'].value_counts().plot(kind='barh',figsize=(5, 12))

Vamos novamente agregar as informações dessa tabela no anterior, que contém os intens e ordens. Podemos usar como chave de unificação o `product_id`

In [ ]:
df = pd.merge(df,df_products,on='product_id')

Agora que fizemos a junção, podemos calcular as categorias mais compradas (antes, calculamos as categorias mais ofertadas)

In [ ]:
df['product_category_name'].value_counts().plot(kind='barh',figsize=(5, 12))

Há correlação entre as categorias ofertadas e as compradas?

In [ ]:
df_products['product_category_name'].value_counts().corr(df['product_category_name'].value_counts())

## Clientes e vendedores

Vamos agora carregar as informações dos vendedores. Essa informação está na tabela `olist_sellers_dataset.csv`

In [ ]:
df_sellers = pd.read_csv("olist_sellers_dataset.csv")

In [ ]:
df_sellers.head()

Na próxima aula faremos uma análise melhor da distribuição geográfica. Neste momento, vamos ver a frequência dos estados dos vendedores

In [ ]:
df_sellers['seller_state'].value_counts().plot(kind='barh',logx=True)

E agora vamos carregar a tabela de consumidores. Essa tabela está no arquivo `olist_customers_dataset.csv`

In [ ]:
df_customers = pd.read_csv("olist_customers_dataset.csv")

In [ ]:
df_customers['customer_state'].value_counts().plot(kind='barh',logx=True)

Vamos agora incluir a informação de consumidores e vendedores na base de dados geral

In [ ]:
df = pd.merge(df, df_sellers, on = 'seller_id')
df = pd.merge(df, df_customers, on = 'customer_id')

Existe uma correspondência entre o estado do vendedor e do comprador? Podemos fazer uma tabela com a frequência cruzada:

In [ ]:
pd.crosstab(df['customer_state'],df['seller_state'])

A tabela ficou um pouco grande, vamos selecionar os 10 estados com o maior número de compradores e vendedores.

In [ ]:
top_sellers_states = df['seller_state'].value_counts().head(10).keys()
top_customer_states = df['customer_state'].value_counts().head(10).keys()

E filtrar a tabela. Vamos ver também normalizado por linha:

In [ ]:
table = pd.crosstab(df['customer_state'],df['seller_state'],normalize=0)

table.style \
  .hide([row for row in table.index if row not in top_customer_states], axis=0) \
  .hide([col for col in table.columns if col not in top_sellers_states], axis=1) \
  .format(precision=3, thousands=".", decimal=",") \
  .background_gradient(axis=0,  cmap="YlGnBu")

Como o preço varia de acordo com o estado do vendedor?

In [ ]:
df[['seller_state','price']].plot(kind='box', by= 'seller_state',logy=True)

Como o frete varia com o estado de comprador?

In [ ]:
df[['seller_state','freight_value']].plot(kind='box', by= 'seller_state',logy=True)

## Distância entre o vendedor e o comprador

O arquivo `olist_geolocation_dataset.csv` tem as coordenadas geográficas do prefixo do CEP:

In [ ]:
df_geo = pd.read_csv('olist_geolocation_dataset.csv', index_col=0)

In [ ]:
df_geo = df_geo.drop_duplicates()

Vamos usar a função `geodesic` da biblioteca `geopy` para calcular a distância geográfica entre o vendedor e o comprador

In [ ]:
def geo_dist_zip(seller_zip_code_prefix, customer_zip_code_prefix):
    try:
        coord_seller = list(df_geo.loc[seller_zip_code_prefix][['geolocation_lat','geolocation_lng']].mean())
        coord_customer = list(df_geo.loc[customer_zip_code_prefix][['geolocation_lat','geolocation_lng']].mean())
        return gp.geodesic(coord_seller,coord_customer).km
    except:
        return

In [ ]:
df['distance'] = df.apply(lambda x: geo_dist_zip(x['seller_zip_code_prefix'],x['customer_zip_code_prefix']),axis=1)

In [ ]:
df['distance'].describe()

In [ ]:
df['distance'].plot(kind='hist',logx=True,density=True,bins=100)
df['distance'].plot(kind='kde', logx=True,xlim=[1,5000])

A distância e o valor do frete estão correlacionados?

In [ ]:
df.plot(kind='scatter', x='distance', y='freight_value')

In [ ]:
df[['distance', 'freight_value']].corr()